In [20]:
# Imports
import unittest as ut

 #Implement Employee Details
with open('Employees.txt','w') as f:
    f.write('''12345 Green Joe 37 16 1.5 72 710''')
    f.close()
#Implement Hours Details 
with open('Hours.txt','w') as f1:
    f1.write('''12345 31/10/2021  42''')
    f1.close()

#create employee class and attributes
class Employee:
    Employees = {}
    Std_rate = 0.2
    Higher_rate = 0.4
    
    def __init__(self, staffId, Firstname, Lastname, Reghours, Hourlyrate, OTMultiple, Taxcredit, Standardband):
        self.Employees = {}   
        self.staffId = int(staffId)
        self.Firstname = str(Firstname)
        self.Lastname = str(Lastname)
        self.Reghours = int(Reghours)
        if Reghours <=0:
            raise ValueError("incorrect value! hours should be more than 0")
        self.Hourlyrate = int(Hourlyrate)
        if Hourlyrate <= 0:
            raise ValueError("incorrect value! hours should be more than 0")
        self.OTMultiple = float(OTMultiple)
        if OTMultiple < 0:
            raise ValueError("incorrect value! hours should be more than 0")
        self.Taxcredit = int(Taxcredit)
        if Taxcredit < 0:
            raise ValueError("incorrect value! hours should be more than 0")
        self.Standardband = int(Standardband)
        if Standardband <= 0:
            raise ValueError("incorrect value! hours should be more than 0")


    def configure(Employeesfile):
        with open(Employeesfile) as f:
            for line in f:
                staffId,Firstname,Lastname,Reghours,Hourlyrate,OTMultiple,Taxcredit,Standardband = line.split()
                Employee.Employees[staffId] = Employee(staffId,Firstname,Lastname,int(Reghours),int(Hourlyrate),float(OTMultiple),int(Taxcredit),int(Standardband))

    def computepayment(self, Hoursworked, Date):
        salary = dict()
        salary['Name'] = self.Firstname + " "
        salary['Date'] = Date
        salary['RHW'] = min(Hoursworked, self.Reghours)#To calculate Regular Hours Worked
        if Hoursworked < 0:
            raise ValueError('hours worked cannot be negative')
        salary['OHW'] = max(0, Hoursworked - self.Reghours)#To calculate OverTime Hours Worked
        salary['Regular rate'] = self.Hourlyrate
        salary['Overtime rate'] = self.Hourlyrate * self.OTMultiple#To calculate Overtime rate
        salary['Regular pay'] = self.Hourlyrate * salary['RHW']#To calculate Regular pay
        salary['Overtime pay'] = max(salary['Overtime rate'] * salary['OHW'], 0)#To calculate Overtime pay
        salary['Gross pay'] = salary['Regular pay'] + salary['Overtime pay']#To calculate Gross pay
        salary['Standard rate pay'] = self.Standardband
        salary['Higher rate pay'] = max(salary['Gross pay'] - self.Standardband, 0)#To calculate Higher rate pay
        salary['Standard tax'] = salary['Standard rate pay'] * 0.2#To calculate Standard tax
        salary['Higher tax'] = round(salary['Higher rate pay'] * 0.4, 2)#To calculate Higher tax
        salary['Total tax'] = round(salary['Standard tax'] + salary['Higher tax'], 2)#To calculate Total tax 
        salary['Tax credit'] = self.Taxcredit
        salary['PRSI'] = salary['Gross pay'] * 0.4#To calculate PRSI
        salary['Net tax'] =round(salary['Total tax'] - salary['Tax credit'], 2)#To calculate Net tax
        salary['Net deductions'] = round(salary['Net tax'] + salary['PRSI'], 2)#To calculate Net deduction
        salary['Net pay'] = max(0, salary['Gross pay'] - salary['Net deductions'])#To calculate Net pay
        if salary['Net pay'] <0:
            raise ValueError("incorrect value! net pay should be more than 0 & cannot be in negative")
        return salary

jg=Employee(12345,'Green','Joe',37,16,1.5,72,710)
jg.computepayment(42,'30/01/2022')

{'Name': 'Green ',
 'Date': '30/01/2022',
 'RHW': 37,
 'OHW': 5,
 'Regular rate': 16,
 'Overtime rate': 24.0,
 'Regular pay': 592,
 'Overtime pay': 120.0,
 'Gross pay': 712.0,
 'Standard rate pay': 710,
 'Higher rate pay': 2.0,
 'Standard tax': 142.0,
 'Higher tax': 0.8,
 'Total tax': 142.8,
 'Tax credit': 72,
 'PRSI': 284.8,
 'Net tax': 70.8,
 'Net deductions': 355.6,
 'Net pay': 356.4}

In [25]:
class TestEmployee(ut.TestCase):

    def test_hoursnotnegative(self):
        jb = Employee(12345,'Green','Joe',37, 16, 1.5, 72, 710)
        with self.assertRaises(ValueError):
            jb.computepayment(-40, '31/10/2021')

    def testnegativeotpay(self):
        jb = Employee(1234,'Green','Joe', 37, 16, 1.5,72,710)
        otpay = jb.computepayment(100, '27/03/2023')
        self.assertGreaterEqual(otpay['Overtime pay'], 0)
        self.assertGreaterEqual(otpay['OHW'], 0)   

    def test_negative_hours_worked(self):
        with self.assertRaises(ValueError):
            jb = Employee(12345,'Green','Joe',37, 16, 1.5, 72, 710)
            jb.computepayment(-1, '30/05/2022')

    def test_negative_regularhours(self):
        with self.assertRaises(ValueError):
            jb = Employee(12345,'Green','Joe',-37, 16, 1.5, 72, 710)
            jb.computepayment(42, '16/09/1998')

    def test_zero_regularhours(self):
        with self.assertRaises(ValueError):
            jb = Employee(12345,'Green','Joe',0, 16, 1.5, 72, 710)
            jb.computepayment(42, '16/09/1998')

    def test_negative_hourlyrate(self):
        with self.assertRaises(ValueError):
            jb = Employee(12345,'Green','Joe',37, -16, 1.5, 72, 710)
            jb.computepayment(42, '16/09/1998')

    def test_zero_hourlyworked(self):
        with self.assertRaises(ValueError):
           jb = Employee(12345,'Green','Joe',37, 0, 1.5, 72, 710)
           jb.computepayment(42, '16/09/1998') 

    def test_negatuve_OTMultiple(self):
        with self.assertRaises(ValueError):
            jb = Employee(12345,'Green','Joe',37, 16, -1.5, 72, 710)
            jb.computepayment(42, '16/09/1998') 

    def testnegative_taxcredit(self):
        with self.assertRaises(ValueError):
            jb = Employee(12345,'Green','Joe',37, 16, 1.5, -72, 710)
            jb.computepayment(42, '16/09/1998') 

    def testnegative_standardnamd(self):
        with self.assertRaises(ValueError):
            jb = Employee(12345,'Green','Joe',37, 16, 1.5, 72, -710)
            jb.computepayment(42, '16/09/1998') 

    def test_zero_standardband(self):
        with self.assertRaises(ValueError):
            jb = Employee(12345,'Green','Joe',37, 16, 1.5, 72, 0)
            jb.computepayment(42, '16/09/1998') 


    def test_negative_overtimepay(self):
        with self.assertRaises(ValueError):
            jb = Employee(12345,'Green','Joe',37, -16, 1.5, 72, 710)
            otp = jb.computepayment(42, '16/09/1998') 
            self.assertGreaterEqual(otp['Overtime pay'], 0)
            self.assertGreaterEqual(otp['OHW'], 0)

In [26]:
ut.main(argv=['ignored'], exit=False)

............
----------------------------------------------------------------------
Ran 12 tests in 0.051s

OK
